# Построение baseline моделей

## 1. Подготовка данных

Импорт необходимых библиотек

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer, Imputer, StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, classification_report, roc_curve
from xgboost import XGBClassifier

Загрузка данных:

In [2]:
train_df = pd.read_csv('orange_small_churn_train_data.csv', index_col='ID')
test_df = pd.read_csv('orange_small_churn_test_data.csv', index_col='ID')

y = train_df['labels'].apply(lambda x: 1 if x==1 else 0)
X = train_df.drop(['labels'], axis=1)

X_train, X_hold, \
y_train, y_hold = train_test_split(X, y,
                                   test_size=0.2,
                                   random_state=9,
                                   shuffle=True,
                                   stratify=y)
X_test = test_df

Реализуем препроцессинг данных.

Данные содержат много пропусков и выбросов, а также константные признаки.
Функция, находящие неконстантные признаки:

In [3]:
def select_features(X_train, y_train):

    # 1. filter out const columns
    vc = X_train.apply(lambda col: len(col.value_counts()))
    vc = vc[vc > 0]
    all_cols = X_train.columns.values
    num_cols = set(all_cols[:190])
    cat_cols = set(all_cols[190:])
    non_const_all_cols = set(vc.index.values)
    non_const_num_cols = sorted(list(non_const_all_cols.intersection(num_cols)))
    non_const_cat_cols = sorted(list(non_const_all_cols.intersection(cat_cols)))

    print('non-const columns:', len(non_const_all_cols))

    # 2. correlation feature selection

    num_corrs = X_train[non_const_num_cols].apply(lambda col: point_biserial_corr(col.values, y_train), axis=0)
    top_corrs = sorted(num_corrs.abs().sort_values(ascending=False)[:100].index)

    all_cols = sorted(list(set(top_corrs).union(non_const_cat_cols)))

    return all_cols, top_corrs, non_const_cat_cols


def point_biserial_corr(x, y):
    y = y[~np.isnan(x)]
    x = x[~np.isnan(x)]
    p = y.mean()
    q = 1 - p
    ex = x.mean()
    sx = x.std(ddof=0)

    px = x[y==1]
    nx = x[y==0]

    return (px.mean() - nx.mean())/sx*math.sqrt(p*q)

Функция, исключающая выбросы:

In [4]:
def filter_outliers(X_train, y_train, cols, alpha):
    print('filtering outliers...')
    for col in cols:
        var = X_train[col]
        var_churn = var[y_train==1]
        var_loyal = var[y_train==0]

        outliers = len(X_train)
        condition = None
        col_a = alpha

        while outliers > 200:
            churn_min, churn_max = var_churn.quantile([col_a, 1 - col_a])
            loyal_min, loyal_max = var_loyal.quantile([col_a, 1 - col_a])

            condition = var.isnull() | \
                        ((y_train==1) & (churn_min <= var) & (var <= churn_max)) | \
                        ((y_train==0) & (loyal_min <= var) & (var <= loyal_max))

            outliers = len(X_train) - len(X_train[condition])
            col_a /= 2

        if condition is not None:
            X_train = X_train[condition]
            y_train = y_train[condition]
    print('finished: ', len(X_train))
    
    return X_train, y_train
    

 Выбросы исключим, а пропуски заполним средними значениями для случая числовых признаков и отдельным признаком для случая категориальных признаков. Кроме того, у некоторых категориальных признаков большое число возможных значений. Ограничим его, взяв только первые 200 наиболее часто встречаемых значения для каждого категориального признака.

Нам потребуется отдельный класс, который будет делать dummy-encoding категориальных переменных с указанными выше особенностями

In [5]:
class DummyEncoder(BaseEstimator, TransformerMixin):
    '''
    Encodes categorical features as one-hot variables with max_categories restriction
    '''
    def __init__(self, columns=None, max_categories=None):
        self.columns = columns
        self.dummy_columns = None
        self.max_categories = max_categories


    def fit(self, X, y=None, **kwargs):
        self.dummy_columns = None
        return self


    def transform(self, X, y=None, **kwargs):
        if self.max_categories is not None:
            X = X[self.columns] if self.columns is not None else X.copy()
            for col in X.columns:
                top_cats = X[col].value_counts()[:self.max_categories].index.values
                X[col] = X[col].apply(lambda x: x if (x in top_cats or x is None) else 'aggr')

        dummy_df = pd.get_dummies(X, columns=self.columns, sparse=True, dummy_na=True)
        new_cols = dummy_df.columns.values
        if self.dummy_columns is None:
            self.dummy_columns = new_cols
            return dummy_df
        else:
            res_df = pd.DataFrame()
            for col in self.dummy_columns:
                res_df[col] = dummy_df[col] if col in new_cols else np.zeros((len(X),), dtype=int)
        return res_df

Так как будет использоваться кросс-валидация, даже baseline модели нужно тренировать в пайплайне, чтобы информация о holdout-выборке не использовалась при обучении. Пайплайн будет:
- Фильтровать выбросы (только для обучающей выборки перед обучением)
- Удалять полностью NaN признаки
- Запонять средними значенийми пропуски числовых переменных
- Масштабировать числовые переменные
- Dummy-кодировать категориальные переменные по описанному выше алгоритму

Реализуем функцию, реализующую препроцессинг и конструирующую пайплайн:

In [6]:
def preprocess_and_get_pipeline(X_train, y_train, alg):

    # filter out outliers
    X_train, y_train = filter_outliers(X_train, y_train, X_train.columns[:190], 0.01)
    
    # select features
    all_cols, num_cols, cat_cols = select_features(X_train, y_train)
    
    pipeline = Pipeline(steps=[
            # get rid of fully NaN columns
            ('filter_out_useless_columns', FunctionTransformer(lambda data: data.loc[:, all_cols], validate=False)),

            # processing
            ('processing', FeatureUnion([

                # numeric features
                ('numeric', Pipeline(steps=[
                    ('selecting', FunctionTransformer(lambda data: data.loc[:, num_cols], validate=False)),
                    ('float_nan_mean', Imputer(strategy='mean')),
                    ('scaling', StandardScaler())
                ])),

                # categorical features
                ('categorical', Pipeline(steps=[
                    ('selecting', FunctionTransformer(lambda data: data.loc[:, cat_cols], validate=False)),
                    ('encoding', DummyEncoder(max_categories=200))
                ]))
            ])),

            #model
            ('model', alg)
        ])

    return X_train, y_train, pipeline

## 2. Моделирование

Будем использовать градиентный бустинг

In [7]:
model = XGBClassifier(learning_rate=0.1, n_estimators=100, n_jobs=-1)
params = {
    'model__learning_rate': [0.1],
    'model__n_estimators':  [120] }

X_train, y_train, pipeline = preprocess_and_get_pipeline(X_train, y_train, model)

cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=9)

grid = GridSearchCV(pipeline, params, scoring='roc_auc', cv=cv, verbose=10)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

model = grid.best_estimator_

filtering outliers...
finished:  25749
non-const columns: 212
Fitting 7 folds for each of 1 candidates, totalling 7 fits
[CV] model__learning_rate=0.1, model__n_estimators=120 ...............


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice.
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars


[CV]  model__learning_rate=0.1, model__n_estimators=120, score=0.775042813220856, total=  46.6s
[CV] model__learning_rate=0.1, model__n_estimators=120 ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   57.3s remaining:    0.0s


[CV]  model__learning_rate=0.1, model__n_estimators=120, score=0.7381874382240585, total=  48.3s
[CV] model__learning_rate=0.1, model__n_estimators=120 ...............


...

[CV]  model__learning_rate=0.1, model__n_estimators=120, score=0.768004630208985, total=  44.0s
[CV] model__learning_rate=0.1, model__n_estimators=120 ...............


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  5.6min remaining:    0.0s


[CV]  model__learning_rate=0.1, model__n_estimators=120, score=0.7520151018109659, total=  44.0s


[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  6.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  6.5min finished


{'model__learning_rate': 0.1, 'model__n_estimators': 120}
0.75457201929


Посчитаем качество на отложенной выборке:

In [8]:
y_pred = model.predict_proba(X_hold)[:, 1]
score = roc_auc_score(y_hold, y_pred)
print('Holdout score:', score)

Holdout score: 0.728070744841


Обучим модель на всей тренировочной выборке с параметрами, полученными выше, и предскажем результаты тестовой:

In [10]:
model = XGBClassifier(learning_rate=0.1, n_estimators=120, n_jobs=-1)
X_train, y_train, pipeline = preprocess_and_get_pipeline(X, y, model)

pipeline.fit(X, y)

y_pred = pipeline.predict_proba(X_test)[:, 1]

res_df = pd.DataFrame(y_pred, columns=['result'])
res_df.index.name = 'ID'
res_df.to_csv('churn_res.csv', sep=',')

filtering outliers...
finished:  33115
non-const columns: 212


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: Mean of empty slice.
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:60: RuntimeWarning: invalid value encountered in double_scalars


# 3. Результат

Private Score: 0.73519

Public Score:  0.69487